In [3]:
import cv2
from PIL import Image
import numpy as np
import os

# Load the pre-trained Haar Cascade face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')



In [4]:
from keras_facenet import FaceNet

face_model = FaceNet()


In [5]:
def get_face(image):
    # Convert the frame to grayscale (needed for face detection)
    #image=cv2.imread(image_path)
    image=np.array(image)
    gray_frame = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces using the Haar Cascade
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Find the largest face based on area (width * height)
    largest_face = max(faces, key=lambda rect: rect[2] * rect[3])  # rect[2] = width, rect[3] = height

    # Extract coordinates of the largest face
    (x, y, w, h) = largest_face
    face_image = image[y:y+h, x:x+w]
    return face_image

In [6]:
def get_face_embedding(image):
    # Convert the image to PIL format, ensure RGB, and resize to 160x160
    face_image_pil = Image.fromarray(image).convert("RGB").resize((160, 160))
    
    # Convert back to NumPy array
    face_image_array = np.array(face_image_pil)

    # Ensure the image has the correct shape (160, 160, 3)
    if face_image_array.shape[-1] != 3:
        face_image_array = np.stack([face_image_array] * 3, axis=-1)

    # Expand dimensions to (1, 160, 160, 3) for batch processing
    face_image_array = np.expand_dims(face_image_array, axis=0)

    # Get the face embedding using the FaceNet model
    embeddings = face_model.embeddings(face_image_array)[0]

    return embeddings

In [7]:
import psycopg2

def make_connection():

    host = 'localhost'  
    database = 'secure environment'
    user = 'postgres'
    password = '****'
    port = '5432'  

    try:
        # Establish a connection to the database
        connection = psycopg2.connect(
            host=host,
            database=database,
            user=user,
            password=password,
            port=port
        )
    
        # Create a cursor object to interact with the database
        cursor = connection.cursor()

        # Execute a sample query
        cursor.execute('SELECT version();')
        db_version = cursor.fetchone()
        print(f"Connected to: {db_version[0]}")

    except Exception as e:
        print(f"Error: {e}")
    return connection

   

In [8]:
con=make_connection()


Connected to: PostgreSQL 16.3, compiled by Visual C++ build 1939, 64-bit


In [53]:
import ipywidgets as widgets

# Input name
name_input = widgets.Text(
    description='Name:',
    placeholder='Enter your name'
)


In [ ]:
display(name_input)

In [ ]:
name_input.value

In [ ]:

from IPython.display import display
from PIL import Image
from io import BytesIO
# Input image file
file_input = widgets.FileUpload(
    accept='image/*',  # Accept only image files
    multiple=False     # Single file only
)
display(file_input)

def handle_upload(change):
    if file_input.value:  # Check if files are uploaded
        for uploaded_file in file_input.value:
            content = uploaded_file['content']
            image = Image.open(BytesIO(content))
            display(image)
            return image

file_input.observe(handle_upload, names='value')

In [57]:
image=handle_upload(change=None)

In [ ]:
embb_vet=get_face_embedding(get_face(image))

In [ ]:
embb_vet.shape

In [ ]:

# Create cursor
cursor = con.cursor()

try:
    # Try inserting the data
    name = name_input.value  # Extract value from the widget
    embedding = embb_vet.tolist()  # Convert numpy array to list
    
    # Start a new transaction block
    cursor.execute("BEGIN;")
    
    # Execute insert query
    cursor.execute(
        "INSERT INTO face_embedding (name, emb_vector) VALUES (%s, %s)",
        (name, embedding)
    )
    
    # Commit the transaction if everything goes well
    con.commit()
    print("Insertion successful!")

except Exception as e:
    # If error occurs, rollback the transaction
    con.rollback()
    print(f"Error: {e}")

finally:
    # Close the cursor and connection
    cursor.close()
    con.close()


In [5]:
con=make_connection()
cursor = con.cursor()

try:
    # Query to fetch name and embedding
    cursor.execute("SELECT name, emb_vector FROM face_embedding WHERE name = %s", ("Johny",))
    row = cursor.fetchone()
    
    if row:
        # Extract the name and the embedding
        name = row[0]
        embedding = row[1]  # The embedding is directly a list of floats
        print(f"Name retrieved: {row[0]}")  # Should display the full name
        print("Embedding:", embedding)
    else:
        print("No matching record found.")

except Exception as e:
    print(f"Error: {e}")

finally:
    # Close the cursor and connection
    cursor.close()
    con.close()


Connected to: PostgreSQL 16.3, compiled by Visual C++ build 1939, 64-bit
Name retrieved: Johny
Embedding: [[-0.0023831427097320557, 0.06407446414232254, -0.028689062222838402, 0.00297544221393764, 0.04595158249139786, -0.010703997686505318, 0.02998366579413414, 0.1302599012851715, 0.03796929121017456, -0.01149502582848072, -0.029662519693374634, -0.020120244473218918, 0.0074545713141560555, 0.02678004465997219, -0.011213554069399834, 0.04103796184062958, -0.00020617595873773098, 0.003937707282602787, -0.04912029579281807, 0.05299690365791321, -0.004239141475409269, 0.006752119865268469, -0.05370635166764259, 0.025579452514648438, 0.059602804481983185, -0.00360752921551466, 0.020085474476218224, 0.02628342993557453, 0.051923900842666626, 0.018587326630949974, 0.0004661632119677961, -0.007059225346893072, -0.03499949723482132, 0.01636393554508686, 0.056330613791942596, 0.013383984565734863, 0.07022127509117126, -0.036197248846292496, 0.037401024252176285, 0.021855942904949188, 0.01231012

In [11]:

from sklearn.metrics.pairwise import cosine_similarity

# Function to compare face embeddings (cosine similarity)
def get_similarity(embedding1, embedding2):
    embedding1 = np.reshape(embedding1, (1, -1))  
    embedding2 = np.reshape(embedding2, (1, -1)) 
    similarity = cosine_similarity(embedding1, embedding2)[0][0]  # Get the similarity value
    return similarity 

In [ ]:
get_similarity(embb_vet,np.array(embedding))

In [22]:
def insert_unknown(embedding,name):
    con=make_connection()
    # Create cursor
    cursor = con.cursor()

    try:
        # Try inserting the data
        name = name_input.value  # Extract value from the widget
        embedding = embb_vet.tolist()  # Convert numpy array to list
        
        # Start a new transaction block
        cursor.execute("BEGIN;")
        
        # Execute insert query
        cursor.execute(
            "INSERT INTO face_embedding (name,emb_vector) VALUES (%s, %s)",
            (name, embedding)
        )
        
        # Commit the transaction if everything goes well
        con.commit()
        print("Insertion successful!")

    except Exception as e:
        # If error occurs, rollback the transaction
        con.rollback()
        print(f"Error: {e}")

    finally:
        # Close the cursor and connection
        cursor.close()
        con.close()

In [14]:
from datetime import datetime
name=""
folder_path="C:/Users/barra/Desktop/CodeSoft/FACE DETECTION ANDRECOGNITION/temp"
count=0

In [16]:
con=make_connection()
# Start video capture (use 0 for the default webcam)
video_capture = cv2.VideoCapture(0) 
ret, previous_frame = video_capture.read()
if not ret:
    print("Failed to grab initial frame")
    video_capture.release()
    exit()
if not video_capture.isOpened():
    print("Error: Could not open video stream or file")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    
    if not ret:
        print("Error: Failed to capture image")
        break

    
    # Convert the frame to grayscale (needed for face detection)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces using the Haar Cascade
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=10, minSize=(30, 30))

    # Draw rectangles around the faces
    for (x, y, w, h) in faces:
        face_image = gray_frame[y:y+h, x:x+w]
        embedded_face=get_face_embedding(face_image)
        threshold=0.6       
        cursor = con.cursor()
        cursor.execute("SELECT name, emb_vector FROM face_embedding")
        row = cursor.fetchall()
            
        for row in row:
            # Extract the name and the embedding
            embedding = row[1]  # The embedding is directly a list of floats
            similarity=get_similarity(embedded_face,np.array(embedding))
            if similarity> threshold:
                color=(0, 255, 0)
                name = row[0]
                print(name)
                print(similarity)
                break
            else:
                name = "Unknown"
                color = (0, 0, 255) # Red for unrecognized
                print(similarity) 
                face=get_face(frame)
                if not os.listdir(folder_path):
                    cv2.imwrite(f"{folder_path}/face_detected.jpg", face)
                else:
                    for filename in os.listdir(folder_path):
                        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                            file_path = os.path.join(folder_path, filename)
                            image = cv2.imread(file_path)  
                            embedded_face_image=get_face_embedding(face)
                            if get_similarity(embedded_face_image,embedded_face) < 0.2 :  
                                count +=1
                                full_path = f"{folder_path}/face_detected_{count}.jpg"
                                cv2.imwrite(full_path,face)
                                print(f"Saved face image to {full_path}")
                

            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)


    # Display the frame with the detected faces
    cv2.imshow('Face Detection', frame)


    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
# Release the capture and close the window
video_capture.release()
cv2.destroyAllWindows()
cursor.close()
con.close()                        


Connected to: PostgreSQL 16.3, compiled by Visual C++ build 1939, 64-bit
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
-0.018001591517172285
-0.17844516304141217
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
-0.17844516304141217
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
-0.17844516304141217
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
-0.015564853240292185
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
-0.015564853240292185
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
-0.17844512742919777
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
0.203814936719134
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
-0.0850785583446168
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
-0.30031839115696
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
-0.30031839115696
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
-0.30031839115696
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
-0.16107201015169703
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
-0.16107201015169703
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
-0.30031833114959616
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
0